# Import libraries

In [1]:
import pandas as pd
import numpy as np

# Import RAW data

In [2]:
filename = "../../data/diccionario/frecuencia_elementos_corpes_1_0.txt"

columns = ["Forma", "Lema", "Categoria", "Frecuencia", "Frec. norm. con signos ort.", "Frec. norm. sin signos ort."]
df = pd.read_csv(filename, delimiter='\t', header=0, on_bad_lines="warn",encoding='utf-8', names=columns, skiprows=[0])

# Get only the first 3 columns
df = df.iloc[:,0:3]
# Cast columns names to unicode and uppercase
df.columns = [x.upper() for x in df.columns]
# Remove rows with NaN values
df = df.dropna()
# Remove rows with empty values
df = df[df["LEMA"] != " "]

df.head()

Skipping line 1162749: expected 6 fields, saw 7

Skipping line 1725934: expected 6 fields, saw 7



,FORMA,LEMA,CATEGORIA
0,de,de,P
1,",",",",Y
2,.,.,Y
3,el,el,T
4,la,el,T


# Visualize some tags see [etiquetario](./docs/etiquetario_RAE_sustantivos_adjetivos.pdf)

In [3]:
# Get the unique values of the column "CATEGORIA"
categories = df["CATEGORIA"].unique()
# Create an empty DataFrame to store the sampled rows
sample_df = pd.DataFrame(columns=df.columns)
# Loop through each category and sample two rows without replacement
for category in categories:
    category_df = df[df["CATEGORIA"] == category]
    if len(category_df) >= 2:
        sampled_rows = category_df.sample(2, replace=False)
        sample_df = pd.concat([sample_df, sampled_rows])
# Reset the index of the resulting DataFrame
sample_df.reset_index(drop=True, inplace=True)
# Show the result
print(sample_df)

                          FORMA                         LEMA CATEGORIA
0                  de acuerdo a                 de acuerdo a         P
1                      RUMBO AL                      rumbo a         P
2                       5.1.1.3                      5.1.1.3         Y
3                       3.3.1.3                      3.3.1.3         Y
4                          unas                           un         T
5                           las                           el         T
6                  aún y cuando                 aún y cuando         C
7                      MIENTRAS                     mientras         C
8                         CUYOS                         cuyo         H
9                         Cuyas                         cuyo         H
10                      USTEDES                        usted         L
11                         Ella                           él         L
12               cromáticamente               cromáticamente         R
13    

# Get only the sustantives and adjectives see [etiquetado](./docs/etiquetario_RAE_sustantivos_adjetivos.pdf)

In [4]:
# Extract only the column "CATEGORIA" which have the values of adjectives and sustantives
# Mirar la documentación en donde se define el tag de los sustantivos y adjetivos
sustantive_tag = "N"
adjective_tag = "A"
adverb_tag = "R"
# Extract from the raw df DataFrame the rows with the tag "N" or "A"
print("Before: ", len(df))
df = df[df["CATEGORIA"].isin([sustantive_tag, adjective_tag, adverb_tag])]
print("After: ", len(df))
df.head()

Before:  2754080
After:  1781630


,FORMA,LEMA,CATEGORIA
19,no,no,R
28,más,más,R
49,No,no,R
51,años,año,N
57,también,también,R


# Split from sustantivo and adjetivo

In [5]:
# Split the dataframe in sustantives and adjectives and sort by alphabetical order in FORMA and reset the index
sustantives_df = df[df["CATEGORIA"] == sustantive_tag][["LEMA", "FORMA"]]
adjectives_df = df[df["CATEGORIA"] == adjective_tag][["LEMA", "FORMA"]]
adverbs_df = df[df["CATEGORIA"] == adverb_tag][["LEMA", "FORMA"]]
# Show the result
print(sustantives_df.head())
print(adjectives_df.head())
print(adverbs_df.head())

      LEMA   FORMA
51     año    años
82   parte   parte
85    vida    vida
90  tiempo  tiempo
94     vez     vez
           LEMA       FORMA
143      grande        gran
145       mayor       mayor
172       nuevo       nuevo
204       mejor       mejor
209  importante  importante
       LEMA    FORMA
19       no       no
28      más      más
49       no       No
57  también  también
58   cuando   cuando


# Filter, clean and delete data from dataframe

In [6]:
import re
from unidecode import unidecode

def clean_word(word):
    # Delete all numbers
    word = re.sub(r"\d+", "", word)
    # Delete all accents
    word = unidecode(word.lower())
    # Remove simbols
    word = re.sub(r"[^a-z0-9ñ]", "", word)
    return word

def clean_and_process_df(df):
    # Remove registers with nan or empty values in the column "FORMA"
    df = df.dropna(subset=["FORMA"])
    # Apply the cleaning function to the "FORMA" and "LEMA" columns
    df["FORMA"] = df["FORMA"].apply(clean_word)
    df["LEMA"] = df["LEMA"].apply(clean_word)
    # Get all the registers with spaces in the column "FORMA" and delete them
    df = df[~df["FORMA"].str.contains(" ")]
    # Get all the registers with spaces in the column "LEMA" and delete them
    df = df[~df["LEMA"].str.contains(" ")]
    # Remove registers with nan or empty values in the column "LEMA"
    df = df.dropna(subset=["LEMA"])
    # Remove duplicates in the column "FORMA"
    df = df.drop_duplicates(subset=["FORMA"])
    # Remove rows where "LEMA" or "FORMA" are empty strings
    df = df[(df["LEMA"] != "") & (df["FORMA"] != "")]
    return df
# Clean the sustantives and adjectives DataFrames
print("Before: ", len(sustantives_df))
sustantives_df = clean_and_process_df(sustantives_df)
print("After: ", len(sustantives_df))
print("Before: ", len(adjectives_df))
adjectives_df = clean_and_process_df(adjectives_df)
print("After: ", len(adjectives_df))
print("Before: ", len(adverbs_df))
adverbs_df = clean_and_process_df(adverbs_df)
print("After: ", len(adverbs_df))

Before:  1631668
After:  1305938
Before:  140478
After:  100412
Before:  9484
After:  6549


#  Delete stopwords in unstructrured data

In [7]:
# Download stopwords in spanish and english from nltk
import nltk
nltk.download('stopwords')
# Import stopwords from nltk
from nltk.corpus import stopwords
# Get the stopwords in spanish
spanish_stopwords = stopwords.words('spanish')
# Get the stopwords in english
english_stopwords = stopwords.words('english')
# Extend into stopwords
stopwords = spanish_stopwords + english_stopwords
# Unidecode the stopwords
stopwords = [unidecode(word) for word in stopwords]
# Delete all aparitions of stopwords in the sustantives and adjectives DataFrames
sustantives_df = sustantives_df[~sustantives_df["FORMA"].isin(stopwords)]
adjectives_df = adjectives_df[~adjectives_df["FORMA"].isin(stopwords)]
adverbs_df = adverbs_df[~adverbs_df["FORMA"].isin(stopwords)]
# Show the result
print(sustantives_df.head())

      LEMA   FORMA
51     ano    anos
82   parte   parte
85    vida    vida
90  tiempo  tiempo
94     vez     vez


[nltk_data] Downloading package stopwords to /home/fulp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Order alphabetical

In [8]:
# Order alphabetically by "LEMA" and "FORMA" and reset the index
sustantives_df = sustantives_df.sort_values(by=["LEMA", "FORMA"]).reset_index(drop=True)
adjectives_df = adjectives_df.sort_values(by=["LEMA", "FORMA"]).reset_index(drop=True)
adverbs_df = adverbs_df.sort_values(by=["LEMA", "FORMA"]).reset_index(drop=True)

# Get structure data

In [9]:
# Show the result
sustantives_df.head(5)
# Save the DataFrames to CSV files in the folder "data/diccionario" as df_structured_sustantivos.csv and df_structured_adjetivos.csv
sustantives_df.to_csv("../../data/diccionario/df_structured_sustantivos.csv", index=False)
adjectives_df.to_csv("../../data/diccionario/df_structured_adjetivos.csv", index=False)
adverbs_df.to_csv("../../data/diccionario/df_structured_adverbios.csv", index=False)

# Get unstructured data

In [10]:
# Get a list with all LEMAS of the sustantives and sort by alphabetical order
sustantives_lemas = list(sustantives_df["LEMA"])
# Get a list with all LEMAS of the adjetives and sort by alphabetical order
adjectives_lemas = list(adjectives_df["LEMA"])
# Get a list with all LEMAS of the adverbs and sort by alphabetical order
adverbs_lemas = list(adverbs_df["LEMA"])
# Show a sample of the sustantives with the format "index - lemma"
for i, lemma in enumerate(sustantives_lemas[:10]):
    print(i, lemma)
# Get a list with all FORMS of the sustantives and sort by alphabetical order
sustantives_forms = list(sustantives_df["FORMA"])
# Get a list with all FORMS of the adjetives and sort by alphabetical order
adjectives_forms = list(adjectives_df["FORMA"])
# Get a list with all FORMS of the adverbs and sort by alphabetical order
adverbs_forms = list(adverbs_df["FORMA"])
# Show a sample of the sustantives with the format "index - form"
for i, form in enumerate(sustantives_forms[:5]):
    print(i, form)
# Save in txt file as list_unstructured_sustantivos.txt and list_unstructured_adjetivos.txt
with open("../../data/diccionario/list_unstructured_sustantivos.txt", "w") as f:
    f.write("\n".join(sustantives_forms))
with open("../../data/diccionario/list_unstructured_adjetivos.txt", "w") as f:
    f.write("\n".join(adjectives_forms))
with open("../../data/diccionario/list_unstructured_adverbios.txt", "w") as f:
    f.write("\n".join(adverbs_forms))

0 12
1 12d
2 2
3 2ddtt2
4 3
5 3mm3
6 a2b2
7 aa
8 aaa
9 aaaa
0 12
1 12d
2 2
3 2ddtt2
4 3


# Test

In [11]:
def get_syntax(words, df_nouns, unstructured_forms_nouns, df_adj, unstructured_forms_adj, estricto=False):
    # Object palabras_dict
    palabras_dict = {}
    # Object palabras_list
    palabras_list = []
    # Initialize nouns array
    nouns = []
    adjectives = []
    # Split by " "
    for word in words.split(" "):
        word = clean_word(word)
        # Verify if is a noun
        noun = get_lemma_df(word, df_nouns, unstructured_forms_nouns)
        adjective = get_lemma_df(word, df_adj, unstructured_forms_adj)
        adjectives, nouns = inference(adjective, noun, estricto, adjectives, nouns)
    palabras_dict["Sustantivos"] = nouns
    palabras_dict["Adjetivos"] = adjectives
    palabras_list = nouns + adjectives
    return palabras_dict, palabras_list

def clasify_estric_mode(adjective, noun, adjective_list, noun_list):
    if adjective is not None:
        adjective_list.append(adjective)
    elif noun is not None:
        noun_list.append(noun)
    return adjective_list, noun_list

def clasify_non_estric_mode(adjective, noun, adjective_list, noun_list):
    if noun is not None:
        noun_list.append(noun)
    if adjective is not None:
        adjective_list.append(adjective)
    return adjective_list, noun_list

def inference(adjective, noun, estricto, adjective_list, noun_list):
    if estricto:
        adjective_list, noun_list = clasify_estric_mode(adjective, noun, adjective_list, noun_list)
    else:
        adjective_list, noun_list = clasify_non_estric_mode(adjective, noun, adjective_list, noun_list)
    return adjective_list, noun_list

def get_lemma_df(word, df, unstructured_forms):
    try:
        word_index = unstructured_forms.index(word)
        return df.iloc[word_index]["LEMA"]
    except ValueError:
        return None  # Handle the case when the word is not found

# Import and clean test data

In [12]:
from functions.tools import load_json
data = load_json('../../data/test/test_palabras.json')
# Now cast data to a DataFrame
test_df = pd.DataFrame(data)
print(test_df.shape)
test_df.head()
# from functions.descripcion import clean_descripcion
import re

list_remove = ["www", "com","http", "https"]

def tokenize_descripcion(text):
    # Remove links (URLs) from the text using regular expressions
    text = re.sub(r'http(s)?:\s+\S+', '', text, flags=re.IGNORECASE)
    # Remove all occurrences of ".es" (case-insensitive)
    text = re.sub(r'\.es', '', text, flags=re.IGNORECASE)
    # Remove all non alpha characters from the text using regular expressions
    text = re.sub(r'[^a-zA-Z ]+', ' ', text, flags=re.IGNORECASE)
    # Remove unnecessary spaces from the text using regular expressions
    text = re.sub(r'\s+', ' ', text, flags=re.IGNORECASE)
    # Cast all words to lowercase
    text = text.lower()
    return text

def create_palabras_column(text):
    # Split the text into a list of words and filter simultaneously
    palabras = [palabra for palabra in text.split(" ") if len(palabra) > 1 and palabra not in list_remove]
    return palabras

def clean_descripcion(df):
    # tokenize the descripcion
    df['descripcion_oferta'] = df['descripcion_oferta'].apply(tokenize_descripcion)
    # Split the text into a list of words
    df['palabras_descripcion_oferta'] = df['descripcion_oferta'].apply(create_palabras_column)
    return df

def list_words(palabras_empleo_texto):
    # print(palabras_empleo_texto)
    return palabras_empleo_texto.lower().split(" ")[:-1]
test_df = clean_descripcion(test_df)
test_df['palabras_empleo_texto'] = test_df['palabras_empleo_texto'].apply(lambda x: list_words(x))

(11123, 5)


# Test

In [13]:
import time
import pandas as pd

def process_and_update_df(df, sustantives_df, sustantives_forms, adjectives_df, adjectives_forms):
    # Create a new column in the DataFrame to store the result
    df["palabras_list_all"] = ""
    df["palabras_dict"] = ""
    # Iterate over the DataFrame and apply the word extraction function
    for index, description in df["descripcion_oferta"].items():
        palabras_dict, palabras_list = get_syntax(description, sustantives_df, sustantives_forms, adjectives_df, adjectives_forms)
        # Save palabras_list in a new column in the DataFrame, insert the full list
        df.at[index, "palabras_list_all"] = palabras_list
        # Save palabras_dict in a new column in the DataFrame
        df.at[index, "palabras_dict"] = palabras_dict
    # Calculate and add the column with words that appear in palabras legacy but not in palabras nuevas
    df["palabras_legacy_minus_nuevas"] = df.apply(lambda row: list(set(row["palabras_empleo_texto"]) - set(row["palabras_list_all"])), axis=1)
    return df

# Function to measure time and apply the processing function
def process_and_measure_time(df, sustantives_df, sustantives_forms, adjectives_df, adjectives_forms):
    start_time = time.time()
    df = process_and_update_df(df, sustantives_df, sustantives_forms, adjectives_df, adjectives_forms)
    print("Time: ", time.time() - start_time)
    return df
# Get the first 5 rows of the DataFrame and save into a new DataFrame
test_df = test_df.iloc[0:5].copy()
# Call the processing function with your DataFrame
test_df = process_and_measure_time(test_df, sustantives_df, sustantives_forms, adjectives_df, adjectives_forms)
test_df.head(5)

Time:  45.399319887161255


,id_puesto_esco_ull,categoria,subcategoria,palabras_empleo_texto,descripcion_oferta,palabras_descripcion_oferta,palabras_list_all,palabras_dict,palabras_legacy_minus_nuevas
0,1634,Atencion a clientes,Atencion al cliente,"[administrativo, persona, reservas, buceo, ges...",buscamos una persona encargada de gestionar la...,"[buscamos, una, persona, encargada, de, gestio...","[persona, encargado, reserva, actividad, buceo...","{'Sustantivos': ['persona', 'encargado', 'rese...","[recepciones, condiciones, facturas, facturaci..."
1,1984,Ventas al detalle,Venta al detalle,"[dependiente, tiendas, centro, comercial, expe...",se busca dependiente para la tienda tezenis en...,"[se, busca, dependiente, para, la, tienda, tez...","[busca, dependiente, tienda, centro, comercial...","{'Sustantivos': ['busca', 'dependiente', 'tien...","[tiendas, idiomas]"
2,719,Recursos humanos,Prevencion de riesgos,"[puentes, grua, normas, sector, metales, homol...",grupo loxamhune empresa lider en el alquiler d...,"[grupo, loxamhune, empresa, lider, en, el, alq...","[grupo, empresa, lider, alquiler, maquinaria, ...","{'Sustantivos': ['grupo', 'empresa', 'lider', ...","[elevadores, formaciones, normas, riesgos, met..."
3,1508,Comercial y ventas,Comercial,"[asesores, comercial, prevencion, riesgos, lab...",antea prevencion es una compania dedicada a la...,"[antea, prevencion, es, una, compania, dedicad...","[antea, prevencion, compania, prevencion, ries...","{'Sustantivos': ['antea', 'prevencion', 'compa...","[formaciones, modelos, riesgos, equipos, forma..."
4,2280,Ingenieros y tecnicos,Electronica y automatica industrial,"[oficial, mantenimiento, electromecanico, agua...",mantenimiento preventivo y correctivo de sist...,"[mantenimiento, preventivo, correctivo, de, si...","[mantenimiento, preventivo, correctivo, sistem...","{'Sustantivos': ['mantenimiento', 'preventivo'...","[repuestos, tratamiento, electrica, electromec..."


In [14]:
def get_reference(descripcion_oferta, df, index, column):
    # Get the accuracy of the words in palabras_empleo_texto that are in descripcion_oferta
    error = 0
    accuracy = 0
    for word in descripcion_oferta:
        if unidecode(word) not in df[column].iloc[index]:
            error += 1
        else:
            accuracy += 1
    return error, accuracy

def show_results(index, test_df):
    # Get list of words in descripcion_oferta
    descripcion_oferta = test_df["descripcion_oferta"].iloc[index].split(" ")
    error, accuracy = get_reference(descripcion_oferta, test_df, index, "palabras_list_all")
    # Show descripcion_oferta
    print("Descripcion oferta: ", test_df["descripcion_oferta"].iloc[index])
    print("Total de palabras en descripción: ", len(descripcion_oferta))
    print("Accuracy - palabras nuevas: ", accuracy)
    print("Error - palabras nuevas: ", error)
    # Show palabras list all
    print("Palabras nuevas: ", test_df["palabras_list_all"].iloc[index], len(test_df["palabras_list_all"].iloc[index]))
    # Show palabras_empleo_texto
    error, accuracy = get_reference(descripcion_oferta, test_df, index, "palabras_empleo_texto")
    print("Accuracy - palabras legacy: ", accuracy)
    print("Error - palabras legacy: ", error)
    print("Palabras legacy: ", test_df["palabras_empleo_texto"].iloc[index], len(test_df["palabras_empleo_texto"].iloc[index]))
    # Show the words in palabras_empleo_texto that are not in descripcion_oferta
    print("Palabras legacy no encontradas en descripción: ", list(set(test_df["palabras_empleo_texto"].iloc[index]) - set(descripcion_oferta)))
# Define the Markdown output file
output_file = "results.md"
show_results(0, test_df)

Descripcion oferta:  buscamos una persona encargada de gestionar las reservas de nuestras actividades de buceo horario de las mismas gestion de alojamientos venta de actividades y demas acciones administrativas tambien se hara cargo de reservas telefonicas y presenciales asi como de supervisar que el centro fisico este en buenas condiciones 
Total de palabras en descripción:  49
Accuracy - palabras nuevas:  13
Error - palabras nuevas:  36
Palabras nuevas:  ['persona', 'encargado', 'reserva', 'actividad', 'buceo', 'horario', 'mismas', 'gestion', 'alojamiento', 'venta', 'actividad', 'demas', 'accion', 'administrativo', 'hara', 'cargo', 'reserva', 'asi', 'centro', 'fisico', 'buenas', 'condicion', 'encargado', 'horario', 'administrativo', 'telefonico', 'presencial', 'fisico', 'bueno'] 29
Accuracy - palabras legacy:  10
Error - palabras legacy:  39
Palabras legacy:  ['administrativo', 'persona', 'reservas', 'buceo', 'gestion', 'alojamientos', 'ventas', 'telefonicas', 'supervisar', 'centro',

In [15]:
def export_results_to_markdown(index, test_df, output_file):
    # Get list of words in descripcion_oferta
    descripcion_oferta = test_df["descripcion_oferta"].iloc[index].split(" ")
    error, accuracy = get_reference(descripcion_oferta, test_df, index, "palabras_list_all")
    
    with open(output_file, "w") as md_file:
        # Write header for the section
        md_file.write(f"**Descripcion oferta:** {test_df['descripcion_oferta'].iloc[index]}\n")
        md_file.write(f"**Total de palabras en descripción:** {len(descripcion_oferta)}\n")
        md_file.write(f"**Accuracy - palabras nuevas:** {accuracy}\n")
        md_file.write(f"**Error - palabras nuevas:** {error}\n")
        md_file.write(f"**Palabras nuevas:** {', '.join(test_df['palabras_list_all'].iloc[index])}\n")
        palabras_not_found = list(set(test_df['palabras_list_all'].iloc[index]) - set(descripcion_oferta))
        md_file.write(f"**Palabras nuevas no encontradas en descripción:** {', '.join(palabras_not_found)}\n")
        
        error, accuracy = get_reference(descripcion_oferta, test_df, index, "palabras_empleo_texto")
        md_file.write(f"**Accuracy - palabras legacy:** {accuracy}\n")
        md_file.write(f"**Error - palabras legacy:** {error}\n")
        md_file.write(f"**Palabras legacy:** {', '.join(test_df['palabras_empleo_texto'].iloc[index])}\n")
        
        palabras_not_found = list(set(test_df['palabras_empleo_texto'].iloc[index]) - set(descripcion_oferta))
        md_file.write(f"**Palabras legacy no encontradas en descripción:** {', '.join(palabras_not_found)}\n")

# Define the Markdown output file
output_file = "results.md"

# Call the function to export the results to Markdown
export_results_to_markdown(1, test_df, output_file)

In [16]:
# Show palabras_legacy_minus_nuevas
# print("Palabras no encontradas: ", test_df["palabras_legacy_minus_nuevas"].iloc[index], len(test_df["palabras_legacy_minus_nuevas"].iloc[index]))

In [17]:
# Get the index of "alojamientos" in sustantives_forms
index = sustantives_forms.index("alojamientos")
print(index)

46157


In [ ]:
# Ejemplo de uso:
offer = """
En España está presente desde hace más de 25 años, con más de 130 oficinas y más de 1.800 Agentes asociados. Seleccionamos asesores inmobiliarios para nuestra oficina en calle Carvajal, con o sin experiencia.
Te ofrecemos tener tu negocio propio con la menor inversión del mercado trabajando en la empresa líder de Canarias es la mejor elección para profesionales como tu para la industria inmobiliaria y sus clientes, a través de la creación de un entorno de trabajo Sinérgico, transformando y profesionalizando esta industria.
Con RE/MAX puedes llegar a lo más alto de la profesión Inmobiliaria.
¿Qué hace un agente asociado RE/MAX?

- Calificar nuevos clientes.
- Estudia el mercado donde trabaja.
- Capta nuevos inmuebles para la venta.
- Elabora planes de marketing para los inmuebles en cartera.
- Atiende y da el seguimiento a las necesidades de sus clientes
- Aconseja financieramente a sus clientes.
- Concreta la venta de los inmuebles en cartera.
- Realiza valoraciones de valor de mercado de los inmuebles.
"""

def test_unitary(offer):
    start_time = time.time()
    palabras_dict, palabras_list = get_syntax(offer, sustantives_df, sustantives_forms, adjectives_df, adjectives_forms, estricto=True)
    print("Time: ", time.time() - start_time)
    return palabras_dict, palabras_list